In [1]:
!pip install --upgrade openai

In [1]:
from openai import OpenAI
import json

# load and set our key
client= OpenAI(api_key=open("key.txt", "r").read().strip("\n"))

## Get_weather

### Step 1: Define the function schema

In [2]:
functions=[
    {
        "name": "get_weather",
        "description": "Get the current weather for a given location",
        "parameters": {
            "type": "object",
            "properties": { # parameter definitions
                "location": {
                    "type": "string",
                    "description": "city name and optionally the country, e.g., 'Boston, United States' "
                               }
                # "unit": {
                #     "type": "string",
                #     "enum": ["celsius", "fahrenheit"],
                #     "description": "Temperature unit"
                #           }
                },
             "required": ["location"]
            }
        }
]

### Step2: Ask GPT and let it decide whether to call the function or not

In [3]:
user_message=[
    {
    "role": "system",
    "content": "You are a helpful assistant.",
    },
    {"role": "user",
    "content": "What's the weather like in Boston today?"
     }
]

In [5]:
completion=client.chat.completions.create(
    model="gpt-4-0613",
    messages=user_message,
    functions=functions,
    function_call={"name":"get_weather"}
)
completion

ChatCompletion(id='chatcmpl-BUJYjgceFSZpj6NDU6p13sCxVDlv5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "location": "Boston, United States"\n}', name='get_weather'), tool_calls=None))], created=1746564725, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=86, total_tokens=99, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
model_message=completion["choices"][0]["message"]

TypeError: 'ChatCompletion' object is not subscriptable

This error indicates that the variable ```completion```is not a dictionary  or a list, but rather a ```ChatCompletion```object. It can be accessed using object <font color='azure'>**attributes**</font> and methods, not direct subscription with square brackets.

In [6]:
model_message=completion.choices[0].message
model_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "location": "Boston, United States"\n}', name='get_weather'), tool_calls=None)

It seems that the model decided to call the get_weather function.

### Step3: Run the actual function

In [7]:
import requests
import json

def get_weather(location):
  url = "https://open-weather13.p.rapidapi.com/city"

  querystring = {"city":"new york","lang":"EN"}

  headers = {
    "x-rapidapi-key": "55380f5857msh2efb42dd6b82303p13fa99jsn40f1fde1c28f",
    "x-rapidapi-host": "open-weather13.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  data=response.json()

  weather={
      "location": location,
      "temperature":data["main"]["temp"],
      "min_temp": data["main"]["temp_min"],
      "max_temp": data["main"]["temp_max"],
      "description": data["weather"][0]["description"],
      "icon": data["weather"]["icon"],
      "feels_like":data["main"]["feels_like"],
      "humidity": data["main"]["humidity"]
  }

  return weather

In [9]:
import requests
import json

def get_weather(location):
  url = "https://weather-api167.p.rapidapi.com/api/weather/current"

  querystring = {"place":"Boston","units":"standard","lang":"en","mode":"json"}

  headers = {
    "x-rapidapi-key": "55380f5857msh2efb42dd6b82303p13fa99jsn40f1fde1c28f",
    "x-rapidapi-host": "weather-api167.p.rapidapi.com",
    "Accept": "application/json"
  }

  response = requests.get(url, headers=headers, params=querystring)
  data=response.json()

  # weather={
  #     "location": location,
  #     "temperature":data["main"]["temp"],
  #     "min_temp": data["main"]["temp_min"],
  #     "max_temp": data["main"]["temp_max"],
  #     "description": data["weather"][0]["description"],
  #     "icon": data["weather"]["icon"],
  #     "feels_like":data["main"]["feels_like"],
  #     "humidity": data["main"]["humidity"]
  # }

  # return weather

In [10]:
# Parse the structured JSON containing the parameters (arguments) to be used
# from the GPT response and converts it into a Python dictionary using json.loads()
func_args=json.loads(model_message.function_call.arguments)
result=get_weather(**func_args)

### Step4: Send result back to GPT

In [11]:
final_completion=client.chat.completions.create(
    model="gpt-4-0613",
    messages=[
        user_message[1], # use only the user's message without that of system.
        model_message,
        {
            "role": "function",
            "name": "get_weather",
            "content": json.dumps(result)
        }
         ]
)
final_completion

ChatCompletion(id='chatcmpl-BUJZ7CwJkOFijIkRw3ufkkiuX9e57', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, I am currently unable to provide the weather updates for Boston at this moment. Please check a reliable weather forecasting website or app to get the most accurate information.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1746564749, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=43, total_tokens=79, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Step5: Get user-friendly response